## <b> NB02 - Raininess Index </b>

My Raininiess Index (RI) will be composed of the variable <b> rain sum </b>

#### This will help me make deductions regarding my two hypotheses:

<b> H0: </b> London is no different than any other generally-rainy city, in terms of raininess

<b> H1: </b> London is as rainy as the movies make it out to be, defined by how it compares to other cities on raininess

In [2]:
## Importing packages
import pandas as pd
import os
import json
import requests
import numpy as np

from functions import *

from lets_plot import *
LetsPlot.setup_html()

import geopandas as gpd

from lets_plot.geo_data import *

import geodatasets

from IPython.display import HTML

The geodata is provided by © OpenStreetMap contributors and is made available here under the Open Database License (ODbL).


In [3]:
## Reading London data and renaming rain_sum as raininess (as a test-run)
daily_rain_df = pd.read_json('../data/daily_rain.json')

daily_rain_df = daily_rain_df.rename(columns={'rain_sum': 'raininess'})

daily_rain_df.to_csv('../data/london_rain.csv', index=False)

In [4]:
## Reading the complete city data and renaming rain_sum as raininess
all_city_rain_df = pd.read_csv('../data/historical_city_rain_data.csv')

all_city_rain_df = all_city_rain_df.rename(columns={'rain_sum': 'raininess'})

all_city_rain_df.to_csv('../data/historical_city_rain_data.csv', index=False)

In [5]:
## Converting the data into a pandas table and calling the first 5 rows
data = pd.read_csv('../data/historical_city_rain_data.csv')

df = pd.DataFrame(data)

df.head()

,country,city,date,raininess
0,GB,London,2021-01-01,0.4
1,GB,London,2021-01-02,0.0
2,GB,London,2021-01-03,0.6
3,GB,London,2021-01-04,2.3
4,GB,London,2021-01-05,2.1


In [6]:
## Calculating the mean raininess of each city, to map out the cities' raininess
df = pd.read_csv('../data/historical_city_rain_data.csv')

mean_raininess_per_city = df.groupby('city')['raininess'].mean().reset_index()

mean_raininess_per_city.columns = ['city', 'mean_raininess']

print(mean_raininess_per_city)

      city  mean_raininess
0    Kyoto        5.004197
1   London        2.016423
2   Munich        2.731934
3     Oslo        2.245985
4  Seattle        2.977190


In [7]:
## Saving mean raininess data
mean_raininess_per_city.to_csv('../data/mean_raininess_per_city.csv', index=False)

In [8]:
## Plotting mean raininess data on a livemap
data = pd.read_csv('/files/ds105a-2024-w06-summative-deyavuz/data/mean_raininess_per_city.csv')

centroids = geocode_cities(data["city"]).get_centroids()

lats = [0 * y for y in range(4)]

p = ggplot() + ggsize(800, 500)

plot = (
    p + 
    geom_livemap(zoom=2.75) +
    geom_hline(aes(yintercept=lats), color='#e0218a', linetype=2, size=1) +
    geom_point(aes(size='mean_raininess', color='city'), 
               show_legend=True,  
               data=data,
               map=centroids, 
               map_join="city", 
               tooltips=layer_tooltips().title("@city"))
)

plot

To analyze the data, I will be ranking the cities by:
- every single day, number 1 in raininess

And then:
- calculating how many times London is number 1

Click [here](https://github.com/lse-ds105/ds105a-2024-w06-summative-deyavuz/blob/58b6e8a429c921efab90234ebc7dba23f7571c37/code/NB03-Rankings.ipynb) to navigate to NB03 - Rankings!